In [1]:
import numpy as np
from random import shuffle, choices

In [2]:
def choice_exp(t_lst0, t, disp=1):
    t_lst = []
    w = np.exp(-np.array(t_lst0)/disp)
    while len(t_lst) < t:
        randomChoice_arr = choices(t_lst0, k = 10*t, weights = w)
        for i in randomChoice_arr:
            if int(i) not in t_lst:
                t_lst.append(int(i))
                if len(t_lst) == t:
                    break
    shuffle(t_lst)
    return t_lst

def choice_2_exp(t_lst0, t, disp1=1, disp2=1, w1=20, w2=1):
    t_lst = []
    w = w1*np.exp(-np.array(t_lst0)/disp1) + w2*np.exp(-np.array(t_lst0)/disp2)
    while len(t_lst) < t:
        randomChoice_arr = choices(t_lst0, k = 10*t, weights = w)
        for i in randomChoice_arr:
            if int(i) not in t_lst:
                t_lst.append(int(i))
                if len(t_lst) == t:
                    break
    shuffle(t_lst)
    return t_lst

def make_sample(N,M,L):
    simple_ranking_point_arr = np.zeros(N)
    index_name_set_dic = {i:str(i)+"th" for i in range(N)} # 作品生成
    t_lst0 = [i for i in range(N)]
    counter_arr = np.zeros((N,N)) + 1
    point_arr = np.zeros((N,N))
    n = N // 5
    
    t_select = choice_exp(t_lst0, n, disp=N/3) # 面白い作品が売れている作品に多いと仮定
    t_select.sort()
    print("selected")
    print(t_select)
    
    for i in range(M):#5000票
        t_real = L*np.random.rand()
        t = int(t_real + 1) #読んだ作品数
        t_lst = choice_2_exp(t_lst0, t, disp1=1, disp2=N/2, w1=20, w2=1) # 売上分布が2つの指数関数の和に乗ると仮定
        
        if t > L*0.9:#玄人だけピックアップ
            for _ in range(t): # //6
                for j in range(t-1):
                    if t_lst[j+1] in t_select and t_lst[j] not in t_select and np.random.choice(a=[0,1], size=1, p=[0.8, 0.2]): # np.random.randint(0,2) はなし
                        t_lst[j+1], t_lst[j] = t_lst[j], t_lst[j+1]
        elif t < L*0.5:#読んだ作品数が少ない半分の人だけピックアップ
            for _ in range(int(t)): # //6
                for j in range(t-2, -1, -1):
                    if t_lst[j+1] not in t_select and t_lst[j] in t_select and np.random.choice(a=[0,1], size=1, p=[0.8, 0.2]): # np.random.randint(0,2) はなし
                        t_lst[j+1], t_lst[j] = t_lst[j], t_lst[j+1]
                        
        simple_ranking_point_arr[t_lst[0]] += 4
        if len(t_lst) >= 2:
            simple_ranking_point_arr[t_lst[1]] += 2
        if len(t_lst) >= 3:
            simple_ranking_point_arr[t_lst[2]] += 1
        
        for j in range(t - 1):
            pibot_t = t_lst[j]
            for k in range(j+1, t):
                point_arr[pibot_t, t_lst[k]] += 1
                point_arr[t_lst[k], pibot_t] -= 1
                counter_arr[pibot_t, t_lst[k]] += 1
                counter_arr[t_lst[k], pibot_t] += 1
    
    point_arr /= 10**(-5) + counter_arr**0.5 # 比較された回数の影響を少し残す。メジャー作品ほどポイントが高くなりやすいように,また、後述の計算で低く算出されやすいため
    
    p_lst = []
    for i,p in enumerate(simple_ranking_point_arr):
        p_lst.append((p,i))
    p_lst.sort(reverse=True)
    print("simple ranking")
    for i in range(N):
        if p_lst[i][1] in t_select:
            print(p_lst[i][1], int(p_lst[i][0]), "*")
        else:
            print(p_lst[i][1], int(p_lst[i][0]))
    print("#############################")
    
    return point_arr, t_select

In [3]:
def make_point_lst(point_arr, t_select, N):
    sum_point_arr = np.sum(point_arr, axis=1)
    point_lst = []
    for i,p in enumerate(sum_point_arr):
        point_lst.append((p,i))
    point_lst.sort(reverse=True)
    print("first step ranking")
    for i in range(N):
        if point_lst[i][1] in t_select:
            print(point_lst[i][1], point_lst[i][0], "*")
        else:
            print(point_lst[i][1], point_lst[i][0])
    print("#############################")
    return point_lst, sum_point_arr

In [4]:
def make_ranking_lst(point_lst, N):
    ranking_lst = []
    for i in range(N):
        ranking_lst.append(point_lst[i][1])
    return ranking_lst

In [5]:
def prepare_for_relank(point_arr, sum_point_arr, N):
    X = sum_point_arr.copy()
    count = 0
    while count < 10**6:
        Y = np.dot(point_arr,X)
        X += Y / (10**8)
        X -= np.sum(X)/N
        count += 1
    return X

In [6]:
def make_new_ranking_lst(X,N):
    point_lst = []
    for i,p in enumerate(X):
        point_lst.append((p,i))
    point_lst.sort(reverse=True)

    new_ranking_lst, Y = [], []
    for i in range(N):
        new_ranking_lst.append(point_lst[i][1])
        Y.append(point_lst[i][0])
    return new_ranking_lst, Y

In [7]:
def print_ranking(ranking_lst, new_ranking_lst, t_select, Y,N):
    for i in range(N):
        if new_ranking_lst[i] in t_select:
            print(ranking_lst[i], new_ranking_lst[i], Y[i], "*")
        else:
            print(ranking_lst[i], new_ranking_lst[i], Y[i])

In [8]:
def run(N,M,L):
    point_arr, t_select = make_sample(N,M,L)
    point_lst, sum_point_arr = make_point_lst(point_arr,t_select,N)
    ranking_lst = make_ranking_lst(point_lst,N)
    X = prepare_for_relank(point_arr, sum_point_arr,N)
    new_ranking_lst, Y = make_new_ranking_lst(X,N)
    print_ranking(ranking_lst, new_ranking_lst, t_select, Y, N)

In [9]:
N = 16 # 作品数
M = 5000 # 5000票
L = N // 2 # 読んだ作品数の上限
run(N,M,L)

selected
[0, 1, 6]
simple ranking
0 8148 *
1 6483 *
2 4379
3 3027
4 1905
5 1511
6 1353 *
7 1043
8 843
9 736
10 695
11 624
12 508
13 507
14 408
15 360
#############################
first step ranking
1 35.8239913672725 *
6 32.847757079556345 *
0 29.891502428434933 *
11 1.51918943315155
10 -1.5976575144981824
13 -4.015879852219825
15 -4.332225298563685
7 -5.104836486274428
3 -5.3497177426823095
8 -8.30350339813122
5 -9.75803776065543
12 -10.401116189292793
4 -11.77238671475544
9 -12.12978496247873
2 -12.614601554174644
14 -14.702692834688644
#############################
1 1 35.24180439731066 *
6 6 33.14354720308353 *
0 0 30.419588999009427 *
11 11 0.8428188359770987
10 10 -2.316221839647939
13 15 -2.5448431402605034
15 13 -4.079630140239368
7 7 -5.288596447714239
3 3 -6.638311414890071
8 8 -7.580479847343064
5 12 -9.223350513506109
12 5 -10.562912692695674
4 4 -12.284121083632924
9 9 -12.309584634634358
2 2 -12.96602446787045
14 14 -13.853683212946015


In [10]:
N = 32 # 作品数
M = 5000 # 5000票
L = N // 2 # 読んだ作品数の上限
run(N,M,L)

selected
[1, 2, 4, 8, 9, 17]
simple ranking
0 5634
1 3782 *
2 3015 *
3 2273
4 1693 *
5 1420
7 1132
6 1132
8 1070 *
9 1051 *
10 886
12 852
11 783
13 758
14 736
15 716
17 671 *
16 623
19 571
20 505
21 503
22 472
18 472
24 429
23 401
26 396
25 352
27 313
29 287
28 269
31 268
30 259
#############################
first step ranking
9 73.42404523986869 *
2 68.83626699200775 *
4 60.42031109702256 *
17 52.61295365793272 *
8 47.194820266445646 *
1 34.748171743815384 *
0 18.979169156014343
19 17.070949855020494
26 4.52649899414691
30 -0.15860111887166894
12 -1.3476116137074037
22 -3.069913594764394
29 -4.997799376830005
10 -5.113963331384856
21 -6.116492815569905
7 -9.916045112712437
5 -10.230688790600702
18 -13.228570379990941
31 -13.462682518789027
14 -13.553388933458644
6 -13.859772617651938
25 -18.79794973772208
16 -18.932669606610936
13 -19.174658669374228
3 -19.639041929549027
15 -20.844307409485356
20 -22.883561782911677
28 -27.359600254963016
11 -31.552401117213076
24 -32.11700507088054


In [11]:
N = 64 # 作品数
M = 5000 # 5000票
L = N // 2 # 読んだ作品数の上限
run(N,M,L)

selected
[0, 1, 4, 7, 8, 12, 16, 23, 24, 27, 54, 59]
simple ranking
0 2648 *
1 2218 *
2 2046
3 1367
4 1078 *
5 958
6 844
7 796 *
9 752
10 737
8 725 *
13 671
16 666 *
11 660
18 653
12 640 *
15 619
24 602 *
17 600
22 583
21 577
23 566 *
14 557
25 535
19 507
26 490
27 483 *
33 475
29 461
28 457
20 449
37 437
32 422
30 401
31 394
34 387
39 363
38 348
35 338
41 331
40 310
36 304
42 301
46 281
45 281
58 263
49 254
51 248
52 243
48 242
55 240
50 240
57 236
44 233
54 232 *
56 229
43 229
47 227
59 212 *
53 182
60 162
61 154
63 150
62 129
#############################
first step ranking
7 146.66817602321697 *
16 123.07590361089191 *
4 121.79013814124093 *
24 120.59738409894867 *
23 118.21183829451266 *
12 108.39563589873208 *
8 104.46331067132529 *
1 90.19896401321805 *
27 82.84541203567991 *
54 81.30344997433613 *
59 78.58169739555345 *
0 78.4114398160844 *
58 35.443883889847356
51 22.429851762649704
42 10.488268316211336
33 9.319617337645862
17 3.3467548527553093
34 2.395704296309446
9 2.02989

In [12]:
N = 128 # 作品数
M = 5000 # 5000票
L = N // 2 # 読んだ作品数の上限
run(N,M,L)

selected
[1, 3, 4, 7, 10, 13, 14, 16, 17, 20, 21, 26, 28, 30, 37, 45, 46, 48, 50, 58, 62, 74, 100, 104, 105]
simple ranking
0 2075
2 1140
1 1000 *
3 635 *
4 570 *
5 530
24 449
6 448
10 444 *
19 434
9 434
8 422
20 419 *
13 419 *
7 414 *
17 411 *
23 399
16 398 *
11 394
14 393 *
18 389
42 376
32 376
29 375
15 370
25 367
33 366
28 365 *
21 364 *
12 364
38 358
22 352
46 345 *
30 344 *
31 341
27 335
26 332 *
49 324
36 320
39 311
34 309
35 296
41 291
53 290
57 289
48 288 *
50 286 *
54 285
44 285
40 285
68 280
60 276
37 272 *
72 271
45 269 *
59 263
47 263
55 255
43 247
52 246
51 244
69 242
75 241
73 240
62 236 *
58 234 *
65 229
70 222
61 221
56 221
66 209
63 209
81 205
67 203
76 197
74 195 *
64 193
79 191
87 183
84 176
71 175
78 171
82 166
88 165
91 164
80 163
77 161
99 160
96 159
105 157 *
97 154
93 147
109 145
98 145
86 144
89 138
94 137
83 137
85 135
90 134
104 133 *
95 133
111 128
92 125
103 122
100 120 *
115 117
107 115
114 112
101 108
126 106
106 105
102 103
124 102
121 98
123 97
116 97


In [13]:
N = 128 # 作品数
M = 5000 # 5000票
L = N // 2 # 読んだ作品数の上限
run(N,M,L)

selected
[0, 1, 6, 10, 11, 13, 16, 18, 19, 21, 25, 27, 36, 38, 40, 43, 48, 64, 66, 72, 96, 102, 105, 114, 117]
simple ranking
0 1266 *
2 1206
1 893 *
3 803
4 652
5 512
7 509
9 482
11 462 *
17 457
8 455
12 454
13 443 *
16 435 *
14 427
10 420 *
15 417
6 416 *
23 403
20 397
25 394 *
21 390 *
18 390 *
27 389 *
26 365
24 360
34 359
35 355
22 353
44 346
39 346
19 345 *
33 342
29 338
38 336 *
31 333
28 330
40 323 *
30 320
49 311
37 311
57 308
43 305 *
41 303
45 300
42 300
36 300 *
54 289
47 288
46 285
51 274
32 272
48 271 *
58 267
50 263
52 262
62 258
56 255
60 254
65 246
61 246
66 240 *
55 236
53 235
70 230
64 225 *
86 220
71 218
72 215 *
68 210
77 206
74 206
80 203
76 203
59 199
67 197
82 193
69 191
63 190
78 187
75 182
85 181
96 179 *
81 172
83 171
90 170
89 169
87 169
79 165
84 164
95 160
105 156 *
88 156
73 154
93 151
91 149
92 148
107 140
94 136
126 135
108 133
102 132 *
112 130
97 129
98 128
113 127
103 127
100 123
116 122
104 119
125 117
99 115
127 114
110 113
101 113
106 111
115 110


In [14]:
N = 128 # 作品数
M = 5000 # 5000票
L = N // 2 # 読んだ作品数の上限
run(N,M,L)

selected
[1, 2, 7, 8, 10, 11, 14, 15, 18, 20, 22, 25, 29, 33, 36, 37, 39, 47, 65, 66, 67, 84, 95, 97, 123]
simple ranking
0 2035
1 948 *
3 859
2 739 *
5 667
4 643
7 490 *
6 488
10 484 *
14 466 *
13 443
12 439
8 428 *
17 425
9 420
31 419
26 418
20 402 *
11 402 *
27 398
30 395
16 394
19 382
28 377
23 370
29 368 *
24 364
21 364
41 360
35 358
15 356 *
18 351 *
32 340
34 325
36 324 *
46 323
45 321
25 320 *
33 318 *
22 318 *
38 312
49 299
47 288 *
39 280 *
40 279
44 276
37 272 *
52 268
59 267
56 266
48 261
43 260
54 255
42 255
57 251
50 248
51 242
70 239
53 237
65 236 *
63 233
73 231
83 229
55 228
71 224
78 223
72 222
64 222
60 222
58 216
66 211 *
61 207
84 204 *
69 202
79 199
75 199
67 197 *
74 190
89 177
62 173
87 172
92 169
95 168 *
80 168
68 165
104 164
90 162
108 159
88 153
105 152
86 152
81 150
106 147
102 143
99 139
97 139 *
77 137
76 134
101 132
98 132
91 132
103 131
100 131
119 130
111 129
96 129
85 124
116 123
93 121
112 120
94 117
125 116
82 116
115 113
107 113
114 111
123 108 *
1

In [15]:
N = 128 # 作品数
M = 5000 # 5000票
L = N // 2 # 読んだ作品数の上限
run(N,M,L)

selected
[0, 1, 3, 4, 5, 6, 8, 10, 11, 14, 15, 16, 17, 21, 22, 24, 26, 41, 42, 43, 50, 64, 89, 91, 99]
simple ranking
2 1260
0 1219 *
1 894 *
3 622 *
18 539
9 536
4 534 *
19 500
7 481
13 460
11 447 *
5 442 *
20 441
12 441
6 436 *
8 416 *
23 413
17 413 *
14 406 *
40 402
31 401
22 401 *
10 399 *
33 382
35 378
28 377
25 366
24 366 *
32 358
27 354
38 344
34 343
47 341
45 341
16 338 *
26 335 *
37 330
39 327
41 326 *
36 319
15 314 *
29 310
46 309
42 309 *
53 306
52 301
21 301 *
50 300 *
54 299
51 297
30 292
43 287 *
60 276
55 276
49 276
59 275
48 263
64 261 *
44 259
56 253
58 252
57 245
67 234
62 232
68 225
71 220
85 219
77 218
65 217
66 212
63 211
61 202
72 201
87 195
80 193
70 189
84 187
82 186
86 185
73 184
75 183
96 181
81 180
78 175
76 172
92 171
93 169
74 161
79 157
104 156
105 152
100 152
88 151
95 150
69 148
98 146
97 146
89 141 *
83 141
110 137
114 128
103 128
112 127
90 127
102 124
94 124
121 123
106 123
91 123 *
108 122
116 121
107 119
113 118
115 115
111 114
109 112
125 107
126 1